# Predict severity using construct-text similarity on suicide risk lexicon




In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
from sklearn.preprocessing import LabelEncoder
import warnings
import datetime
from skopt import BayesSearchCV # had to replace np.int for in in transformers.py
from sklearn import metrics

# TODO: !pip install construct-tracker
sys.path.append('./../construct-tracker/src/')
sys.path.append('./../construct-tracker/src/construct_tracker/')

from construct_tracker.machine_learning import feature_importance
from construct_tracker.machine_learning.metrics_report import cm, classification_report, regression_report
from construct_tracker.machine_learning.pipelines import get_params, get_pipelines, get_combinations
import srl_constructs

In [ ]:
location = 'local' 

if location == 'colab':
  from google.colab import drive
  project_name = 'concept_tracker'
  drive.mount('/content/drive')
  input_dir = f'/content/drive/MyDrive/datum/{project_name}/data/ctl/'
  output_dir = f'/content/drive/MyDrive/datum/{project_name}/data/output/lexicon_paper/'
elif location == 'openmind':
  input_dir = '/nese/mit/group/sig/projects/dlow/ctl/'
  output_dir = '/home/dlow/datum/lexicon/data/output/mpnet/'
elif location =='local':
  input_dir = './data/input/ctl/'
  output_features_dir = './data/input/ctl/'
  output_dir = './data/output/ml_performance/cts/'
  

os.makedirs(output_features_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

In [ ]:
import srl_constructs

constructs = srl_constructs.constructs_in_order
constructs_max = [n+'_max' for n in constructs]

In [ ]:
# Define the function with a threshold argument
def transform_value(x, threshold):
    integer_part = np.floor(x)
    decimal_part = x - integer_part
    if decimal_part < threshold:
        return integer_part
    return x

In [ ]:
output_features_dir

In [ ]:

# # better embeddingsa


# X_train_df_sim = pd.read_csv(output_features_dir+"cts-scores_count-False_thresh-0_train10_train_30perc_text_y_balanced_regression_24-08-16T19-48-36/cts_scores.csv")
# X_test_df_sim = pd.read_csv(output_features_dir+"cts-scores_count-False_thresh-0_train10_test_15perc_text_y_regression_24-08-16T21-51-23/cts_scores.csv")


# counts_dir = 'suicide_risk_lexicon_v1.0_counts_and_matches_24-08-15T19-04-55/'

# # count only prototypes
# X_train_df_counts = pd.read_csv(output_features_dir+counts_dir+'suicide_risk_lexicon_v1.0_counts_train10_train_30perc_text_y_balanced_regression.csv')
# X_test_df_counts = pd.read_csv(output_features_dir+counts_dir+'suicide_risk_lexicon_v1.0_counts_train10_test_15perc_text_y_regression.csv')


In [ ]:
# all-mini-lm-v6
counts_dir  = 'suicide_risk_lexicon_v1.0_counts_and_matches_24-08-15T19-04-55/'

X_train_df_counts = pd.read_csv(output_features_dir+counts_dir+'suicide_risk_lexicon_v1.0_counts_train10_train_30perc_text_y_balanced_regression.csv')
X_test_df_counts = pd.read_csv(output_features_dir+counts_dir+'suicide_risk_lexicon_v1.0_counts_train10_test_15perc_text_y_regression.csv')

# # count only prototypes
X_train_df_sum = pd.read_csv(output_features_dir+'cts-scores_count-sum_thresh-03_24-08-14T16-48-28/train10_train_30perc_text_y_balanced_regression_cts-scores.csv')
X_test_df_sum = pd.read_csv(output_features_dir+'cts-scores_count-sum_thresh-03_train10_test_15perc_text_y_regression_24-08-14T22-50-58/cts_scores.csv')

X_train_df_sim = X_train_df_sum.copy()
X_train_df_sim[constructs_max] = X_train_df_sum[constructs_max].mod(1)
X_test_df_sim = X_test_df_sum.copy()
X_test_df_sim[constructs_max] = X_test_df_sum[constructs_max].mod(1)

# import matplotlib.pyplot as plt
# plt.hist(X_test_df_sum[constructs_max].values.ravel(), bins = 2000)
# plt.xlim((-1, 2))


In [ ]:
# # Check if they match in rows and values to the concat. 

# assert X_train_df_sim.shape[0] == X_train_df_counts.shape[0]
# assert X_test_df_sim.shape[0] == X_test_df_counts.shape[0]

# assert X_train_df_sim['conversation_id'].tolist() == X_train_df_counts['conversation_id'].tolist()
# assert X_test_df_sim['conversation_id'].tolist() == X_test_df_counts['conversation_id'].tolist()



In [ ]:
X_train_df_sum = X_train_df_sim.copy()
X_train_df_sum[constructs_max] = X_train_df_sum[constructs_max].values+X_train_df_counts[constructs].values

X_test_df_sum = X_test_df_sim.copy()
X_test_df_sum[constructs_max] = X_test_df_sum[constructs_max].values+X_test_df_counts[constructs].values

X_test_df_sum

In [ ]:
plt.hist(X_test_df_sum[constructs_max].values.ravel(), bins = 2000)
plt.xlim((-1, 2))

create different datasets:

- thresholds: 0, 0.3, 0.45, 0.6, 0.75
- counts: sim, counts, round1

In [ ]:
# datasets = {
# 	'counts': [X_train_df_counts, X_test_df_counts, constructs],
# 	'sim' : [X_train_df_sim, X_test_df_sim, constructs_max],
# 	'sum': [X_train_df_sum, X_test_df_sum, constructs_max],
# }

In [ ]:
# dataset = 'sim'
# threshold_value = 0

In [ ]:


# # Apply the function to each element in the DataFrame with a threshold argument


# if dataset == 'sum_all_tokens':

# 	# remove counts, just similarites
# 	X_train_df[srl_constructs.constructs_in_order] = X_train_df[srl_constructs.constructs_in_order].mod(1)

# 	# add new counts
# 	X_train_df[srl_constructs.constructs_in_order] =X_train_df[srl_constructs.constructs_in_order].values+ X_train_df_counts[srl_constructs.constructs_in_order].values


# 	# remove counts, just similarites
# 	X_test_df[srl_constructs.constructs_in_order] = X_test_df[srl_constructs.constructs_in_order].mod(1)

# 	# add new counts
# 	X_test_df[srl_constructs.constructs_in_order] = X_test_df[srl_constructs.constructs_in_order].values+ X_test_df_counts[srl_constructs.constructs_in_order].values


# lbl2vec

In [ ]:
# !pip install --upgrade spacy

In [ ]:
from construct_tracker import lexicon
# srl = lexicon.load_lexicon(name = 'srl_v1-0')
srl_prototypes = lexicon.load_lexicon(name = 'srl_prototypes_v1-0')


In [ ]:
import random 

counts_dir  = 'suicide_risk_lexicon_v1.0_counts_and_matches_24-08-15T19-04-55/'

X_train_df_counts = pd.read_csv(output_features_dir+counts_dir+'suicide_risk_lexicon_v1.0_counts_train10_train_30perc_text_y_balanced_regression.csv')
X_test_df_counts = pd.read_csv(output_features_dir+counts_dir+'suicide_risk_lexicon_v1.0_counts_train10_test_15perc_text_y_regression.csv')

In [ ]:
from lbl2vec import Lbl2TransformerVec
from sentence_transformers import SentenceTransformer


toy = False

lexicon_dict = srl_prototypes.to_dict() 

if toy:
	
	lexicon_dict = dict(random.sample(list(lexicon_dict.items()), 2))
constructs = list(lexicon_dict.keys())

	
# select sentence-tranformers model
transformer_model = SentenceTransformer('all-MiniLM-L6-v2') #"all-mpnet-base-v2"


datasets = {}

for split, name in [
	(X_train_df_counts, 'train'),
	(X_test_df_counts, 'test'),

]:

	# init model
	if toy: 
		documents = split['document'].values[:10]
	else:
		documents = split['document'].values

	model = Lbl2TransformerVec(transformer_model=transformer_model,
							keywords_list=list(lexicon_dict.values()), # iterable list of lists with descriptive keywords of type str
							documents=documents, #iterable list of strings
							label_names = list(lexicon_dict.keys())
							)

	# train model
	model.fit()

	similarity_df = model.predict_model_docs() # Predict label similarities for documents used for training
	# similarity_df = model.predict_model_docs(doc_idxs = [0,1]) # Predict label similarities for documents used for training

	# split_sum = split.copy()
	
	# datasets[split] = [split, similarity_df, split_sum]
	split_sim = split.copy()
	split_sim[constructs] = similarity_df[constructs]

	split_sum = split.copy()
	split_sum[constructs] += similarity_df[constructs]
	
	datasets[name] = [split, similarity_df, split_sim, split_sum]
	# model.save(output_dir + name)
	# model = Lbl2Vec.load('model_name')








# choose dataset 

In [ ]:

# X_train_df, X_test_df, constructs_cols = datasets.get(dataset)
# X_train_df, X_test_df, constructs_cols = X_train_df_sim, X_test_df_sim, constructs_max
# X_train_df, X_test_df, constructs_cols = datasets.get(dataset), constructs
split_train, similarity_df_train, split_sim_train, split_sum_train = datasets.get('train')
split_test, similarity_df_test, split_sim_test, split_sum_test = datasets.get('test')
constructs_cols = constructs
X_train_df = split_sim_train.copy()
X_test_df = split_sim_test.copy()
threshold_value = 0

# Clean up column names

# X_train_df.columns = [x.replace('_max', '') for x in X_train_df.columns]
# X_test_df.columns = [x.replace('_max', '') for x in X_test_df.columns]

# Keep as DF with column names
X_train = X_train_df[constructs_cols]
X_test = X_test_df[constructs_cols]

y_train = X_train_df['y'].values
y_test = X_test_df['y'].values





if threshold_value:
	X_train = X_train.applymap(lambda x: transform_value(x, threshold_value))
	X_test = X_test.applymap(lambda x: transform_value(x, threshold_value))


# Just the decimel
# X_train.mod(1)

# just the integer

# X_train = X_train.applymap(np.floor).astype(int)


# Models

In [ ]:
toy = False

# Config
balance = True # balance training set by downsampling
task = 'regression'
target = 'immiment_risk' # [1,2,3]
normalize_lexicon = True

if task == 'classification':
	dv = 'suicide_ladder_classification'
	if target == 'suicidal_desire':
		balance_values = ['nonsuicidal','suicidal_desire']
	elif target == 'imminent_risk':
		balance_values = ['suicidal_desire','imminent_risk']
	smallest_value = 'imminent_risk'
	n = 1893

elif task == 'regression':

	# config
	dv = 'suicide_ladder_a'
	balance_values = [1,2,3]
	smallest_value = 3

feature_vectors = [
	# 'all-MiniLM-L6-v2', 
	# 'srl_validated',
	# 'srl_unvalidated',
	# 'SRL GPT-4 Turbo', 
	# 'liwc22', 
	# 'liwc22_semantic'
	'cts_thresh-03_count-sum',
	] # srl_unvalidated_text_descriptives','text_descriptives' ]

sample_sizes = ['all'] # # TODO: sample_sizes = ['all', 150] 

if task == 'classification':
	scoring = 'f1'
	metrics_to_report = 'all'
	model_names = ['LGBMRegressor'] # model_names = ['LGBMRegressor', 'LogisticRegression'] 
	
elif task == 'regression':
	scoring = 'neg_mean_squared_error'
	# metrics_to_report = ['Model','n', 'RMSE','RMSE per value','MAE','MAE per value',  'rho', 'gridsearch', 'Best parameters']
	model_names = ['LGBMRegressor']  # TODO: 'Ridge']
	metrics_to_report = 'all'

gridsearch = True#, 'minority'
balance = True

In [ ]:

# for similarity_threshold in [None, 0.3, 0.45, 0.6] # Missing None
# for lexicon in [srl, srl_prototypes, just examples] 
# for count_if_exact_match in [False,  'replace', 'sum', 'sum1']
# this can be done with the same features dataset.
# for count_if_exact_match in [just decimal,  just int, 'sum', if>0=1, else nothing]

# CTS similarities 
# CTS replace with count
# CTS sum (similarites + count): if this doesnt work, others might not work
# start with prototype
# CTS sum 1 (similarites + 1 if exact match): need to implement: > 1 ==1 (easy), might give best result



In [ ]:


np.random.seed(123)

# TODO: see where to save feature_vector (tfidf, liwc22) and where to save model_name (Ridge, LightGBM)



ts_i = datetime.datetime.utcnow().strftime('%y-%m-%dT%H-%M-%S')

if toy:
	sample_sizes = [150]
	feature_vectors = feature_vectors[:2]

for n in sample_sizes:
	results = []
	# for gridsearch in [True]:

	# for feature_vector in ['srl_unvalidated', 'all-MiniLM-L6-v2']:#['srl_unvalidated']:#, 'srl_unvalidated']:
	for feature_vector in feature_vectors:#['srl_unvalidated']:#, 'srl_unvalidated']:
		output_dir = output_dir+feature_vector+'/'
		os.makedirs(output_dir, exist_ok=True)

		if toy:
			output_dir_i = output_dir + f'results_{ts_i}_toy/'
		else:
			output_dir_i = output_dir + f'results_{ts_i}_{n}_{task}_{balance_values[-1]}/'
			
		os.makedirs(output_dir_i, exist_ok=True)
		
	
		if toy:
			
			X_train['y'] = y_train
			X_train = X_train.sample(n = 100)
			y_train = X_train['y'].values
			X_train = X_train.drop('y', axis=1)
	
		elif n!='all':
			X_train['y'] = y_train
			X_train = X_train.sample(n = n, random_state=123)
			y_train = X_train['y'].values
			X_train = X_train.drop('y', axis=1)
	

		if task == 'classification':
			encoder = LabelEncoder()

			# Fit and transform the labels to integers
			y_train = encoder.fit_transform(y_train)
			y_test = encoder.transform(y_test)

		
		for model_name in model_names: 
	
			pipeline = get_pipelines(feature_vector, model_name = model_name)
			print(pipeline)
		
			# I'd need y_val 

			# if gridsearch == 'minority':
			# 	# Obtain all hyperparameter combinations
			# 	parameters = get_params(feature_vector,model_name=model_name, toy=toy)
			# 	parameter_set_combinations = get_combinations(parameters)
			# 	scores = {}
			# 	for i, set in enumerate(parameter_set_combinations):
			# 		pipeline.set_params(**set)
			# 		pipeline.fit(X_train,y_train)
			# 		y_pred = pipeline.predict(X_val) # validation set 
			# 		rmse_per_value = []
			# 		rmse = metrics.mean_squared_error(y_val, y_pred, squared=False ) # validation set 
			# 		for value in np.unique(y_val):
			# 			y_pred_test_i = [[pred,test] for pred,test in zip(y_pred,y_val) if test == value] # validation set 
			# 			y_pred_i = [n[0] for n in y_pred_test_i]
			# 			y_test_i = [n[1] for n in y_pred_test_i]
			# 			rmse_i = metrics.mean_squared_error(y_test_i, y_pred_i, squared=False )
			# 			rmse_per_value.append(rmse_i )
			# 		scores[i] = [rmse]+rmse_per_value+[str(set)]
			# 	scores = pd.DataFrame(scores).T
			# 	scores.columns = ['RMSE', 'RMSE_2', 'RMSE_3', 'RMSE_4', 'Parameters']
			# 	scores = scores.sort_values('RMSE_4')
			# 	best_params = eval(scores['Parameters'].values[0])
			# 	pipeline.set_params(**best_params)
			# 	pipeline.fit(X_train,y_train)
			# 	y_pred = pipeline.predict(X_test)
				
			if gridsearch == True:
				parameters = get_params(feature_vector,model_name=model_name, toy=toy)
	
				pipeline = BayesSearchCV(pipeline, parameters, cv=5, scoring=scoring, return_train_score=False,
				n_iter=32, random_state=123)    
				if feature_vector != 'tfidf':
					if 'y' in X_train.columns:
						warnings.warn('y var is in X_train, removing')
						X_train = X_train.drop('y', axis=1)
						
						
				pipeline.fit(X_train,y_train)
				best_params = pipeline.best_params_
				best_model = pipeline.best_estimator_
				if feature_vector != 'tfidf':
					if 'y' in X_test.columns:
						warnings.warn('y var is in X_test, removing')
						X_test = X_test.drop('y', axis=1)
				y_pred = best_model.predict(X_test)
			else:
				pipeline.fit(X_train,y_train)
				best_params = 'No hyperparameter tuning'
				y_pred = pipeline.predict(X_test)
			
			# Predictions
			# output_filename = f'{feature_vector}_{model_name}_{classes_i[1]}_{n}_clauses-{amount_of_clauses}'
			# custom_cr, sklearn_cr, cm_df_meaning, cm_df, cm_df_norm, y_pred_df = metrics_report.save_classification_performance(y_test_i, y_pred, y_proba_1, output_dir_i, output_filename=output_filename,feature_vector=feature_vector, model_name=model_name,best_params = best_params, classes = classes_i,amount_of_clauses=amount_of_clauses, save_output=True)
			
			y_pred_df = pd.DataFrame(y_pred)
			y_pred_df.to_csv(output_dir_i+f'y_pred_{feature_vector}_{model_name}_gridsearch-{gridsearch}_{n}_{ts_i}.csv', index=False)
			path = output_dir_i + f'scatter_{feature_vector}_{model_name}_gridsearch-{gridsearch}_{n}_{ts_i}'
		
			# Performance
			if task == 'classification':
				cm_df_meaning, cm_df, cm_df_norm = cm(y_test,y_pred, output_dir_i, model_name, ts_i, classes = balance_values, save=True)
				y_proba = pipeline.predict_proba(X_test)       # Get predicted probabilities
				y_proba_1 = y_proba[:,1]
				y_pred = y_proba_1>=0.5*1                   # define your threshold
				results_i = classification_report(y_test, y_pred, y_proba_1, output_dir_i,gridsearch=gridsearch,
										best_params=best_params,feature_vector=feature_vector,model_name=model_name,round_to = 2, ts = ts_i)
			elif task == 'regression':

				results_i =regression_report(y_test,y_pred,y_train=y_train,
										metrics_to_report = metrics_to_report,
											gridsearch=gridsearch,
										best_params=best_params,feature_vector=feature_vector,model_name=model_name, plot = True, save_fig_path = path,n = n, round_to = 2)
			results_i.to_csv(output_dir_i + f'results_{feature_vector}_{model_name}_gridsearch-{gridsearch}_{n}_{ts_i}.csv')
			display(results_i)
			results.append(results_i)
			results_df = pd.concat(results)
			results_df = results_df.reset_index(drop=True)
			results_df.to_csv(output_dir_i + f'results_{n}_{ts_i}.csv', index=False)
		
			# Feature importance
			if feature_vector == 'tfidf':
				if model_name in ['XGBRegressor']:
					warnings.warn('Need to add code to parse XGBoost feature importance dict')
				else:
					feature_importances = feature_importance.tfidf_feature_importances(pipeline, top_k = 50, savefig_path = output_dir_i + f'feature_importance_{feature_vector}_{model_name}_{n}_{ts_i}')
			else:
				feature_names = X_train.columns
				feature_importance_df = feature_importance.generate_feature_importance_df(pipeline, model_name,feature_names,  xgboost_method='weight', model_name_in_pipeline = 'model')
				if str(feature_importance_df) != 'None':       # I only implemented a few methods for a few models
					feature_importance_df.to_csv(output_dir_i + f'feature_importance_{feature_vector}_{model_name}_gridsearch-{gridsearch}_{n}_{ts_i}.csv', index = False)        
					# display(feature_importance.iloc[:50])
			
		
			# NaN analysis
			if type(X_train) == pd.core.frame.DataFrame:
				df = X_train.copy()
				# Find the column and index of NaN values
				nan_indices = df.index[df.isnull().any(axis=1)].tolist()
				nan_columns = df.columns[df.isnull().any()].tolist()
				# print("Indices of NaN values:", nan_indices)
				print("Columns with NaN values:", nan_columns)
				print(df.size)
				nans = df.isna().sum().sum()
				print('% of nans:', np.round(nans/df.size,3))
			
	
			